In [110]:
import os
os.listdir()

['.config',
 'NLPholybnycha',
 'notebooks',
 'docs',
 'requirements.txt',
 'raw.csv',
 'drive',
 'raw (1).csv',
 '.git',
 'data',
 'raw (2).csv',
 'src',
 'notebooks_lab2_cleaning_normalization.ipynb',
 'tests',
 'labs',
 'sample_data']

In [111]:
import pandas as pd

# Відкриваємо raw.csv
df_raw = pd.read_csv("raw.csv")
df_raw.head()  # подивимося перші 5 рядків

,split,sent_id,token_id,token,lemma,upos,head,deprel
0,train,ParlaMint-UA_2003-10-14-m0.u1.p1.lang1.s1,1,Доброго,добрий,ADJ,2,amod
1,train,ParlaMint-UA_2003-10-14-m0.u1.p1.lang1.s1,2,ранку,ранок,NOUN,0,root
2,train,ParlaMint-UA_2003-10-14-m0.u1.p1.lang1.s1,3,",",",",PUNCT,6,punct
3,train,ParlaMint-UA_2003-10-14-m0.u1.p1.lang1.s1,4,шановні,шановний,ADJ,6,amod
4,train,ParlaMint-UA_2003-10-14-m0.u1.p1.lang1.s1,5,народні,народний,ADJ,6,amod


In [112]:
# Групуємо токени по sent_id
sentences_df = df.groupby("sent_id")["token"].apply(lambda x: " ".join(x)).reset_index()
sentences_df.rename(columns={"token": "raw_text"}, inplace=True)
sentences_df.head(5)

,sent_id,raw_text
0,NSDC_UA_28_Feb2014-1,"Шановні колеги , Рада національної безпеки має..."
1,NSDC_UA_28_Feb2014-10,"Рішень , які захистять Україну , які зможуть л..."
2,NSDC_UA_28_Feb2014-100,Наших — це в основному військово-морські сили ...
3,NSDC_UA_28_Feb2014-101,А крім морських сил у нас немає ніяких сухопут...
4,NSDC_UA_28_Feb2014-102,15 тисяч — це повний склад військових .


In [113]:
# =============================
# 0. Встановлюємо бібліотеки
# =============================
!pip install pandas

# =============================
# 1. Імпорти
# =============================
import os
import pathlib
import shutil
import pandas as pd
import json
import sys

# =============================
# 2. Створюємо структуру репозиторію
# =============================
folders = [
    "data/raw",
    "data/processed_v2",
    "data/sample",
    "docs",
    "src",
    "tests",
    "notebooks",
    "labs/lab02"
]

for f in folders:
    pathlib.Path(f).mkdir(parents=True, exist_ok=True)

# =============================
# 3. Завантажте ваш raw.csv у Colab
#    і вкажіть правильний шлях нижче
# =============================
from google.colab import files

uploaded = files.upload()  # з’явиться кнопка вибору файлу
# Припустимо, ви завантажили raw.csv у Files Colab
shutil.copy("/content/raw.csv", "data/raw/raw.csv")


Saving raw.csv to raw (3).csv


'data/raw/raw.csv'

In [114]:
preprocess_code = """
import re

URL_RE = re.compile(r'https?://\\S+|www\\.\\S+')
EMAIL_RE = re.compile(r'\\S+@\\S+')
PHONE_RE = re.compile(r'\\+?\\d[\\d\\s\\-()]{7,}')
MULTISPACE_RE = re.compile(r'\\s+')

def clean_text(text: str) -> str:
    text = str(text).replace("\\xa0", " ")
    text = MULTISPACE_RE.sub(" ", text)
    return text.strip()

def normalize_text(text: str) -> str:
    text = text.replace("’","'").replace("ʼ","'")
    text = text.replace("—","-")
    text = text.replace("“",'"').replace("”",'"')
    return text

def mask_pii(text: str) -> str:
    text = URL_RE.sub("<URL>", text)
    text = EMAIL_RE.sub("<EMAIL>", text)
    text = PHONE_RE.sub("<PHONE>", text)
    return text

def preprocess(text: str) -> str:
    text = clean_text(text)
    text = normalize_text(text)
    text = mask_pii(text)
    return text
"""

with open("src/preprocess.py", "w", encoding="utf-8") as f:
    f.write(preprocess_code)

In [115]:
sys.path.append("src")
from preprocess import preprocess

df = pd.read_csv("data/raw/raw.csv")  # Ваш UD CSV
# Групуємо токени по sent_id в речення
sentences_df = df.groupby("sent_id")["token"].apply(lambda x: " ".join(x)).reset_index()
sentences_df.rename(columns={"token": "raw_text"}, inplace=True)
sentences_df.head(5)

,sent_id,raw_text
0,NSDC_UA_28_Feb2014-1,"Шановні колеги , Рада національної безпеки має..."
1,NSDC_UA_28_Feb2014-10,"Рішень , які захистять Україну , які зможуть л..."
2,NSDC_UA_28_Feb2014-100,Наших — це в основному військово-морські сили ...
3,NSDC_UA_28_Feb2014-101,А крім морських сил у нас немає ніяких сухопут...
4,NSDC_UA_28_Feb2014-102,15 тисяч — це повний склад військових .


In [116]:
sentences_df["clean_text"] = sentences_df["raw_text"].apply(preprocess)
sentences_df.head(5)

,sent_id,raw_text,clean_text
0,NSDC_UA_28_Feb2014-1,"Шановні колеги , Рада національної безпеки має...","Шановні колеги , Рада національної безпеки має..."
1,NSDC_UA_28_Feb2014-10,"Рішень , які захистять Україну , які зможуть л...","Рішень , які захистять Україну , які зможуть л..."
2,NSDC_UA_28_Feb2014-100,Наших — це в основному військово-морські сили ...,Наших - це в основному військово-морські сили ...
3,NSDC_UA_28_Feb2014-101,А крім морських сил у нас немає ніяких сухопут...,А крім морських сил у нас немає ніяких сухопут...
4,NSDC_UA_28_Feb2014-102,15 тисяч — це повний склад військових .,15 тисяч - це повний склад військових .


In [117]:
# Зберігаємо тільки очищені речення
sentences_df.to_csv("data/processed_v2/dataset_clean.csv", index=False)

# Малий sample для перевірки
sample_size = min(50, len(sentences_df))
sentences_df.sample(sample_size).to_csv("data/sample/sample.csv", index=False)

In [118]:
# =============================
# 8. Генеруємо audit_summary_lab2.md (довжина речень)
# =============================
raw_len = sentences_df["raw_text"].str.len().mean()
clean_len = sentences_df["clean_text"].str.len().mean()
changed_pct = (sentences_df["raw_text"] != sentences_df["clean_text"]).mean() * 100

audit_text = f"""
# Аудит Lab2

Середня довжина речення до: {raw_len:.2f} символів
Середня довжина речення після: {clean_len:.2f} символів
Відсоток змінених речень: {changed_pct:.2f}%
"""

with open("docs/audit_summary_lab2.md", "w", encoding="utf-8") as f:
    f.write(audit_text)

print("✅ Аудит Lab2 з довжинами речень створено!")

✅ Аудит Lab2 з довжинами речень створено!


In [119]:
policy_text = """
# Політика препроцесингу

1. Нормалізація пробілів
2. Нормалізація апострофів (’ → ')
3. Нормалізація тире (— → -)
4. Нормалізація лапок (“ ” → ")
5. Маскування PII: URL, EMAIL, PHONE
6. Детермінований, відтворюваний pipeline
7. Sentence splitting: не розбивати після скорочень UA (м., т.д.) та чисел/версій
"""
with open("docs/preprocess_policy.md", "w", encoding="utf-8") as f:
    f.write(policy_text)

In [120]:
edge_cases = []

for i, row in sentences_df.iterrows():
    text = row["clean_text"]
    if any(x in row["raw_text"] for x in ["’","ʼ","—","“","”","http","www","@", "+", "м.","т.д."]):
        edge_cases.append({
            "id": str(row["sent_id"]),
            "raw_text": row["raw_text"],
            "clean_text": row["clean_text"],
            "expected_behavior": "текст очищений/нормалізований/замаскований"
        })
    if len(edge_cases) >= 20:
        break

with open("tests/edge_cases.jsonl", "w", encoding="utf-8") as f:
    for case in edge_cases:
        f.write(json.dumps(case, ensure_ascii=False) + "\n")

In [121]:
readme_text = """
# Lab02 Очистка та нормалізація

## Завдання
Детермінований препроцесинг речень з UD CSV.

## Як запускати
1. Відкрити notebook lab2_cleaning_normalization.ipynb
2. Запустити всі клітинки
3. Результат буде в data/processed_v2, data/sample, docs, tests/lab02

## Зміни після Lab2
- Речення очищені, нормалізовані, PII замасковано
- Згенеровано processed_v2 dataset
- Створено невеликий sample
- Згенеровано аудит summary
- Додано edge cases
"""
with open("labs/lab02/README.md", "w", encoding="utf-8") as f:
    f.write(readme_text)

print("✅ Lab02 pipeline виконано успішно!")

✅ Lab02 pipeline виконано успішно!


In [122]:
import os

# ===============================
# 1️⃣ Створюємо dataset_card.md
# ===============================
dataset_card_path = "/content/docs/dataset_card.md"
os.makedirs("/content/docs", exist_ok=True)

# Рахуємо кількість файлів у папках
raw_count = len(os.listdir("/content/data/raw")) if os.path.exists("/content/data/raw") else 0
processed_count = len(os.listdir("/content/data/processed_v2")) if os.path.exists("/content/data/processed_v2") else 0
sample_count = len(os.listdir("/content/data/sample")) if os.path.exists("/content/data/sample") else 0

dataset_card_content = f"""
# Dataset Card

## Джерело
Raw CSV з токенами UD.

## Очищення
1. Видалено зайві пробіли.
2. Нормалізовано апострофи та тире.
3. Лапки приведено до стандартного формату.
4. Маскування PII: URL, EMAIL, PHONE.
5. Результат збережено в data/processed_v2/dataset_clean.csv.

## Sample
Малий sample (50 записів) доступний в data/sample/sample.csv.

## Ризики
- Деякі токени можуть містити некоректні символи після маскування.
- Розбиття речень не враховує контекст скорочень.
"""

with open(dataset_card_path, "w", encoding="utf-8") as f:
    f.write(dataset_card_content.strip())

print(f"✅ Створено {dataset_card_path}")


# ===============================
# 2️⃣ Створюємо requirements.txt
# ===============================
requirements_path = "/content/requirements.txt"

# Список імпортів з твоїх скриптів та ноутбука
# Для прикладу – основні бібліотеки, реально використовувані у preprocess.py та ноутбуку
requirements_content = """
pandas>=2.0
numpy>=1.25
"""

with open(requirements_path, "w", encoding="utf-8") as f:
    f.write(requirements_content.strip())

print(f"✅ Створено {requirements_path}")

✅ Створено /content/docs/dataset_card.md
✅ Створено /content/requirements.txt


In [127]:
# 🚀 Автоматичний пуш потрібних файлів на GitHub

import subprocess
import os

# ===============================
# 1️⃣ Налаштування репозиторію
# ===============================
repo_path = "/content"  # твоя робоча папка
os.chdir(repo_path)

# Ініціалізація git (якщо ще не зроблено)
if not os.path.exists(".git"):
    subprocess.run(["git", "init"])
    print("✅ Git репозиторій ініціалізовано.")

# ===============================
# 2️⃣ Додати remote (замінити на свій токен/юзер)
# ===============================
github_url = "https://@github.com/RitaHolubnycha/NLPholybnycha.git"

# Додаємо або оновлюємо remote origin
try:
    subprocess.run(["git", "remote", "add", "origin", github_url], check=True)
    print("✅ Remote origin додано.")
except subprocess.CalledProcessError:
    subprocess.run(["git", "remote", "set-url", "origin", github_url])
    print("✅ Remote origin оновлено.")

# ===============================
# 3️⃣ Файли, які потрібно додати
# ===============================
files_to_add = [
    "data/raw",
    "data/processed_v2",
    "data/sample",
    "docs/dataset_card.md",
    "docs/preprocess_policy.md",
    "docs/audit_summary_lab2.md",
    "notebooks/lab2_cleaning_normalization.ipynb",
    "src/preprocess.py",
    "tests/edge_cases.jsonl",
    "labs/lab02/README.md",
    "requirements.txt"
]

# Додаємо файли, якщо вони існують
for f in files_to_add:
    if os.path.exists(f):
        subprocess.run(["git", "add", f])
        print(f"✅ Додано {f}")
    else:
        print(f"⚠️ Файл не знайдено: {f}")

# ===============================
# 4️⃣ Коміт
# ===============================
try:
    subprocess.run(["git", "commit", "-m", "Initial commit from Colab"], check=True)
    print("✅ Коміт створено.")
except subprocess.CalledProcessError:
    print("ℹ️ Немає змін для коміту.")

# ===============================
# 5️⃣ Пуш на GitHub
# ===============================
try:
    subprocess.run(["git", "branch", "-M", "main"], check=True)
    subprocess.run(["git", "push", "-u", "origin", "main"], check=True)
    print("✅ Файли успішно запушено на GitHub!")
except subprocess.CalledProcessError as e:
    print("❌ Помилка при пуші:", e)

✅ Remote origin оновлено.
✅ Додано data/raw
✅ Додано data/processed_v2
✅ Додано data/sample
✅ Додано docs/dataset_card.md
✅ Додано docs/preprocess_policy.md
✅ Додано docs/audit_summary_lab2.md
⚠️ Файл не знайдено: notebooks/lab2_cleaning_normalization.ipynb
✅ Додано src/preprocess.py
✅ Додано tests/edge_cases.jsonl
✅ Додано labs/lab02/README.md
✅ Додано requirements.txt
ℹ️ Немає змін для коміту.
❌ Помилка при пуші: Command '['git', 'push', '-u', 'origin', 'main']' returned non-zero exit status 128.
